In [2]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
from pykalman import KalmanFilter
import statistics
import seaborn as sns; sns.set(style="whitegrid")
import math

%matplotlib inline

In [10]:
data = pd.read_csv('datasets/data_cart.csv', index_col=0)
data.shape

(390, 71)

In [ ]:
data = pd.read_csv('datasets/data_cart.csv', index_col=0)
data.shape

In [13]:
y=data['ENBR3']
x= data['FLRY3']

x = sm.add_constant(x)
model = sm.OLS(y, x, window=50).fit()

In [14]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  ENBR3   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.593
Method:                 Least Squares   F-statistic:                     567.8
Date:                Thu, 30 Jul 2020   Prob (F-statistic):           5.75e-78
Time:                        20:58:17   Log-Likelihood:                -629.46
No. Observations:                 390   AIC:                             1263.
Df Residuals:                     388   BIC:                             1271.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.7504      0.416     21.010      0.0

(378, 71)